In [ ]:
from collections import deque

import numpy as np

import torch

In [ ]:
class Agent:
  def __init__(self):
    pass
  def get_state():
    pass
  def remember(self, state, action, reward, next_state, done): #It's likelu we will need to change this {done} to be something which is linked to my project like the final token or something
    pass
  def train_long_memory(self):
    pass
  def train_short_memory(self):
    pass
  def get_action(self, state):
    pass
  def train():
    pass